In [ ]:
!git clone https://github.com/Musab1Blaser/LLM-Neo.git
!cd LLM-Neo && pip install -e ".[torch,metrics]"

In [ ]:
!pip install deepspeed

In [ ]:
import os

# Inject environment flag so DeepSpeed won't complain
os.environ["FORCE_TORCHRUN"] = "1"

In [ ]:
!huggingface-cli login --token <hf token>

In [1]:
# !echo 'export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True' >> ~/.bashrc
# !source ~/.bashrc
export FORCE_TORCHRUN=1

In [ ]:
!cd LLM-Neo && python3 script_Neo.py

In [ ]:
!cd LLM-Neo && bash run_train.sh

In [ ]:
!huggingface-cli repo create your-repo-name --type model

In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="your-username/your-repo-name",
    folder_path="./your_folder",
    path_in_repo="",  # optional
    repo_type="model",  # or "dataset"
    commit_message="Uploading folder contents"
)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

In [ ]:
# Step 1: Load PEFT config
peft_model_id = "/LLM-Neo/saves/llama3-3b/lora/neo/final_model/"
peft_config = PeftConfig.from_pretrained(peft_model_id)

print("here")
# Step 2: Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="auto",
    torch_dtype="auto"
)
print("here")

# Step 3: Load PEFT adapter
model = PeftModel.from_pretrained(base_model, peft_model_id)
print("here")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)
print("here")


In [ ]:
model

In [ ]:
# Optional: Merge PEFT adapter with base model (useful for inference)
model = model.merge_and_unload()

In [ ]:
repo_name = "llama_3b_sparse_neo" # Choose a name for your repository
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)